In [1]:
#Assignment 2


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://apache.mirrors.hoobly.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [4]:
!ls

data.csv       mycsv.csv    spark-2.4.5-bin-hadoop2.7	     task_1.csv
drive	       new2	    spark-2.4.5-bin-hadoop2.7.tgz
file.csv       new.csv	    spark-2.4.5-bin-hadoop2.7.tgz.1
find_text.csv  sample_data  spark-2.4.5-bin-hadoop2.7.tgz.2


In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

In [6]:
!ls 

data.csv       mycsv.csv    spark-2.4.5-bin-hadoop2.7	     task_1.csv
drive	       new2	    spark-2.4.5-bin-hadoop2.7.tgz
file.csv       new.csv	    spark-2.4.5-bin-hadoop2.7.tgz.1
find_text.csv  sample_data  spark-2.4.5-bin-hadoop2.7.tgz.2


In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("groupbyagg").getOrCreate()
from pyspark.sql import SQLContext    
from pyspark.sql.types import DataType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as func
import pandas as pd



In [10]:
#reading and importing data to a Spark Dataframe
df = spark.read.csv("data.csv", inferSchema = True, header = True)
#Filtering out columns for analysis
df = df.select('id_str', 'tweet_created_at', 'user_verified', 'favorite_count', 'retweet_count', 'text_')
#Step 1: Remove the records where “user_verified” is “FALSE”. 
df1 = df.where(df.user_verified=="True")
df1.count()       

# Number of records = 171797

171797

In [0]:
#Cleaning the datetime column to extract day
df2 = df1.withColumn('day', df['tweet_created_at'].substr(5,6))
df2 = df2.drop(df2.tweet_created_at)
df2 = df2.withColumnRenamed("day","tweet_created_at")
#Step 2: For the remaining records (“user_verified” is “TRUE”), group by created date, and count the number of tweets for each date. 
data_counts = df2.groupby("tweet_created_at").count()
max_value = data_counts.select(func.max('count')).collect()[0][0]
result = data_counts.filter(data_counts["count"] == max_value)
created_date = result.collect()[0][0]
print(created_date)   #Jan 03
df3 = df2[df2.tweet_created_at == created_date]
#For the date with highest number of tweets calculate the sum
#of “favorite_count” and “retweet_count” for each tweet on that day, and find out the text content
#(“text_”) of the top 100 tweets with highest sum
df3 = df3.withColumn("favorite_count", df3["favorite_count"].cast(IntegerType()))
df3 = df3.withColumn("retweet_count", df3["retweet_count"].cast(IntegerType()))
df3 = df3.withColumn('total_col', df3.favorite_count + df3.retweet_count).sort('total_col', ascending=False).limit(100)
df3 = df3.select("text_")
df3.show(n=100)


In [0]:
#Create a csv to export top 100 tweets from a Spark Dataframe to a csv
df3 = df3.withColumn('word', func.explode(func.split(func.col('text_'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)
df3.write.csv('task_1.csv',header= "TRUE")

In [0]:
#Task 2: Joining mytask.csv and Amazon data to a single dataframe 
#using spark and exporting the results. The joins were done on id_str
text_df = spark.read.csv("find_text.csv", inferSchema = True, header = True)
text_df = text_df.select("id_str")
text_df.count()
text_combined = text_df.join(df,"id_str","left")
text_combined = text_combined.select("id_str","text_")
text_combined.show()
#Convert to csv and export file
text_combined.coalesce(1).write.option("header", "true").csv("sample_file.csv")

----------------------------------------------------------